In [3]:
import numpy as np 
import pandas as pd
import json 
import csv
import nltk
import unicodedata
import re
import sys
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim, spacy
from gensim.models.ldamulticore import LdaMulticore
import re
from gensim.models import Phrases
from gensim.models.phrases import Phraser

import os

In [6]:
# pip install git+https://github.com/rwalk/gsdmm.git

  Cloning https://github.com/rwalk/gsdmm.git to /private/var/folders/d0/5_1bb8ld291843m4x7vj7vnw0000gn/T/pip-req-build-4pcp086d
  Running command git clone -q https://github.com/rwalk/gsdmm.git /private/var/folders/d0/5_1bb8ld291843m4x7vj7vnw0000gn/T/pip-req-build-4pcp086d
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Created wheel for gsdmm: filename=gsdmm-0.1-py3-none-any.whl size=4603 sha256=3715c0516cb8799c01ca93efc652613cc60f02888dce751fce509fb8ce246a7e
  Stored in directory: /private/var/folders/d0/5_1bb8ld291843m4x7vj7vnw0000gn/T/pip-ephem-wheel-cache-cqhlbodh/wheels/81/2c/23/3ff788bcc6063bf30116ad1a06e75d3ba9aad3f7bc4aba765b
Successfully built gsdmm
Note: you may need to restart the kernel to use updated packages.


In [7]:
from gsdmm import MovieGroupProcess

In [39]:
df = pd.read_csv("../pos_neg_usa.csv")

In [41]:
df.drop(columns = ['Unnamed: 0', 'index', 'new_text', 'languages', 'new_text_after_translation', 'pos_tagged_words'], inplace = True)
df

,Text,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,want find way end racism try force people inje...,"['want', 'find', 'way', 'end', 'racism', 'try'...","['want', 'find', 'way', 'end', 'racism', 'try'...","['way', 'end', 'racism', 'force', 'people', 'b...",subjective,2
1,"this child almost died tetanus, spent months h...","['child', 'almost', 'died', 'tetanus', 'spent'...","['child', 'almost', 'die', 'tetanus', 'spend',...","['child', 'tetanus', 'month', 'hospital', 'cos...",subjective,0
2,juste se rappeler m. leblanc la pression que l...,"['remember', 'leblanc', 'pressure', 'provincia...","['remember', 'leblanc', 'pressure', 'provincia...","['leblanc', 'pressure', 'business', 'world', '...",subjective,0
3,"you would expect cases rise significantly, cou...","['would', 'expect', 'case', 'rise', 'significa...","['would', 'expect', 'case', 'rise', 'significa...","['case', 'vaccination', 'rate', 'play', 'space...",subjective,2
4,depends whether vaccination status is embedded...,"['depends', 'whether', 'vaccination', 'status'...","['depend', 'vaccination', 'status', 'embed', '...","['vaccination', 'status', 'code', 'link', 'con...",subjective,2
...,...,...,...,...,...,...
315598,pretty remarkable achievement. thank you all v...,"['pretty', 'remarkable', 'achievement', 'thank...","['pretty', 'remarkable', 'achievement', 'thank...","['achievement', 'volunteer', 'shotsshotsshot',...",subjective,2
315599,oh that explains similar - seemingly out nowhe...,"['explains', 'similar', 'seemingly', 'nowhere'...","['explain', 'similar', 'seemingly', 'nowhere',...","['message', 'background', 'folk', 'vaccinesave...",subjective,0
315600,do it others.... 💪🎉💉🤙 #vaccinesaveslives,"['others', 'vaccinesaveslives']","['other', 'vaccinesaveslive']","['other', 'vaccinesaveslive']",subjective,2
315601,yay 🙌🏻 congrats dr. j being fully vaccinated i...,"['yay', 'congrats', 'fully', 'vaccinated', 'be...","['congrat', 'fully', 'vaccinate', 'good', 'fee...","['congrat', 'feeling', 'safetyfirst', 'vaccine...",subjective,2


In [42]:
df['words_processed_noun'].values[0][0]

'['

In [43]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [44]:
tokens_words_processed_noun = list(sent_to_words(df['words_processed_noun']))

In [45]:
df['words_processed_noun'] = tokens_words_processed_noun

In [46]:
df['words_processed_noun'].values[0][0]

'way'

# GSDMM for the topic modeling: 

In [47]:
np.random.seed(0)

In [48]:
df['words_processed_noun'].iloc[224817]

['health', 'choice', 'other', 'risk', 'exposure', 'system', 'herd_immunity']

In [49]:
df.info() # no empty rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315603 entries, 0 to 315602
Data columns (total 6 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Text                               315603 non-null  object
 1   words_processed_all                315603 non-null  object
 2   words_processed_noun_adj_verb_adv  315603 non-null  object
 3   words_processed_noun               315603 non-null  object
 4   text_type                          315603 non-null  object
 5   sentiment                          315603 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 14.4+ MB


In [50]:
df['sentiment'].value_counts()

2    222780
0     92823
Name: sentiment, dtype: int64

In [51]:
positive_df = df[df['sentiment'] == 2]

In [52]:
positive_df

,Text,words_processed_all,words_processed_noun_adj_verb_adv,words_processed_noun,text_type,sentiment
0,want find way end racism try force people inje...,"['want', 'find', 'way', 'end', 'racism', 'try'...","['want', 'find', 'way', 'end', 'racism', 'try'...","[way, end, racism, force, people, bunch, peopl...",subjective,2
3,"you would expect cases rise significantly, cou...","['would', 'expect', 'case', 'rise', 'significa...","['would', 'expect', 'case', 'rise', 'significa...","[case, vaccination, rate, play, space, adult, ...",subjective,2
4,depends whether vaccination status is embedded...,"['depends', 'whether', 'vaccination', 'status'...","['depend', 'vaccination', 'status', 'embed', '...","[vaccination, status, code, link, connect, end...",subjective,2
5,it could have ripped… harder or do you think m...,"['could', 'ripped', 'harder', 'think', 'maskin...","['could', 'rip', 'hard', 'think', 'mask', 'iso...","[isolation, question, look, spike, matter, vac...",subjective,2
6,they do this at vaccination centres as well. t...,"['vaccination', 'center', 'well', 'refused', '...","['vaccination', 'center', 'well', 'refuse', 'l...","[vaccination, center, parent, mask, amp, amp, ...",subjective,2
...,...,...,...,...,...,...
315597,"contrary american belief, vaccine is not perso...","['contrary', 'american', 'belief', 'vaccine', ...","['contrary', 'vaccine', 'personal', 'choice', ...","[vaccine, choice, people, risk, vaccine, part]",subjective,2
315598,pretty remarkable achievement. thank you all v...,"['pretty', 'remarkable', 'achievement', 'thank...","['pretty', 'remarkable', 'achievement', 'thank...","[achievement, volunteer, shotsshotsshot, fauci...",subjective,2
315600,do it others.... 💪🎉💉🤙 #vaccinesaveslives,"['others', 'vaccinesaveslives']","['other', 'vaccinesaveslive']",[other],subjective,2
315601,yay 🙌🏻 congrats dr. j being fully vaccinated i...,"['yay', 'congrats', 'fully', 'vaccinated', 'be...","['congrat', 'fully', 'vaccinate', 'good', 'fee...","[congrat, feeling, safetyfirst]",subjective,2


In [53]:
positive_df['words_processed_all'].values[0][0]

'['

In [54]:
positive_tokenised_lemmatized = list(sent_to_words(positive_df['words_processed_all']))

In [55]:
positive_tokenised_lemmatized[0][0]

'want'

In [56]:
len(positive_tokenised_lemmatized)

222780

In [57]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster,sort_dicts))

### Cluster size: 4

In [58]:
mgp = MovieGroupProcess(K=4, alpha=0.1, beta=0.2, n_iters=30)

vocab = set(x for review in positive_tokenised_lemmatized for x in review)
n_terms = len(vocab)
model = mgp.fit(positive_tokenised_lemmatized, n_terms)

In stage 0: transferred 159315 clusters with 4 clusters populated
In stage 1: transferred 126370 clusters with 4 clusters populated
In stage 2: transferred 64092 clusters with 4 clusters populated
In stage 3: transferred 41222 clusters with 4 clusters populated
In stage 4: transferred 33640 clusters with 4 clusters populated
In stage 5: transferred 31111 clusters with 4 clusters populated
In stage 6: transferred 29819 clusters with 4 clusters populated
In stage 7: transferred 29371 clusters with 4 clusters populated
In stage 8: transferred 29008 clusters with 4 clusters populated
In stage 9: transferred 28927 clusters with 4 clusters populated
In stage 10: transferred 28788 clusters with 4 clusters populated
In stage 11: transferred 28769 clusters with 4 clusters populated
In stage 12: transferred 28520 clusters with 4 clusters populated
In stage 13: transferred 28530 clusters with 4 clusters populated
In stage 14: transferred 28644 clusters with 4 clusters populated
In stage 15: trans

In [59]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 10)

Number of documents per topic : [41915 52917 48689 79259]

Most important clusters (by number of docs inside): [3 1 2 0]

Cluster 3 : [('vaccinated', 40985), ('not', 26859), ('vaccine', 25124), ('get', 22642), ('people', 19030), ('mask', 15529), ('vaccinate', 11522), ('mandate', 10789), ('couid', 10727), ('anti', 10125)]

Cluster 1 : [('get', 31069), ('pfizer', 23031), ('modern', 19102), ('shot', 15856), ('booster', 15019), ('vaccine', 12967), ('dose', 10937), ('shoot', 9965), ('day', 9175), ('two', 9077)]

Cluster 2 : [('vaccinated', 27830), ('vaccine', 23679), ('couid', 17822), ('not', 16421), ('get', 15948), ('people', 14126), ('vaccinate', 10800), ('effective', 7549), ('one', 6656), ('still', 5945)]

Cluster 0 : [('vaccine', 24214), ('pfizer', 20951), ('modern', 9662), ('not', 8940), ('get', 6463), ('amp', 5721), ('one', 5614), ('dose', 4054), ('people', 3478), ('couid', 3406)]


In [60]:
# I don`t rename the clusters

topic_dict = {}
topic_names = ['type 1',
               'type 2',
               'type 3',
               'type 4'
              ]
for i, topic_num in enumerate(top_index):
    topic_dict[topic_num]=topic_names[i] 

In [61]:
def create_topics_dataframe(data_text=positive_df.Text,  mgp=mgp, threshold=0.3, topic_dict=topic_dict, lemma_text=positive_tokenised_lemmatized):
    result = pd.DataFrame(columns=['Text', 'Topic', 'Lemma-text'])
    for i, text in enumerate(data_text):
        result.at[i, 'Text'] = text
        result.at[i, 'Lemma-text'] = lemma_text[i]
        prob = mgp.choose_best_label(positive_tokenised_lemmatized[i])
        if prob[1] >= threshold:
            result.at[i, 'Topic'] = topic_dict[prob[0]]
        else:
            result.at[i, 'Topic'] = 'Other'
    return result

In [ ]:
result = create_topics_dataframe(data_text=positive_df.Text, mgp=mgp, threshold=0.3, topic_dict=topic_dict, lemma_text=positive_tokenised_lemmatized)
result.head(5)

In [ ]:
import plotly.express as px

fig = px.pie(result, names='Topic',  title='Topics', color_discrete_sequence=px.colors.sequential.Burg)
fig.show()

In [ ]:
result['Topic']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
types_counts = result.Topic.value_counts()
fig, ax = plt.subplots(1, 2, figsize=(15,5))
types = sns.barplot(x = types_counts.index, y = types_counts.values, palette="pastel", ax=ax[1])


In [ ]:
result['len'] = result.Text.apply(lambda row: len(row.split()))
result.head()

In [ ]:
result['Lemma_text'] = result['Lemma-text'].apply(lambda row: ' '.join(row))

In [ ]:
result = result.drop('Lemma-text', axis=1)

In [ ]:
from wordcloud import WordCloud 

In [ ]:
def create_WordCloud(data, title=None):
    wordcloud = WordCloud(width = 500, height = 500,
                          background_color ='white',
                          min_font_size = 15, collocations=False
                         ).generate(" ".join(data.values))
                      
    plt.figure(figsize = (5, 5), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=20)
    plt.show() 

In [ ]:
create_WordCloud(result['Lemma_text'].loc[result.Topic == 'type 1'], title="Most used words in topic 1")

In [ ]:
create_WordCloud(result['Lemma_text'].loc[result.Topic == 'type 2'], title="Most used words in topic 2")

In [ ]:
create_WordCloud(result['Lemma_text'].loc[result.Topic == 'type 3'], title="Most used words in topic 3")

In [ ]:
create_WordCloud(result['Lemma_text'].loc[result.Topic == 'type 4'], title="Most used words in topic 4")

In [ ]:
create_WordCloud(result['Lemma_text'].loc[result.Topic == 'Other'], title="Most used words in Other")

### Cluster size: 5

In [ ]:
mgp = MovieGroupProcess(K=5, alpha=0.1, beta=0.2, n_iters=30)

vocab = set(x for review in positive_tokenised_lemmatized for x in review)
n_terms = len(vocab)
model = mgp.fit(positive_tokenised_lemmatized, n_terms)

In [ ]:
mgp = MovieGroupProcess(K=6, alpha=0.1, beta=0.2, n_iters=30)

vocab = set(x for review in positive_tokenised_lemmatized for x in review)
n_terms = len(vocab)
model = mgp.fit(positive_tokenised_lemmatized, n_terms)